reference:https://keras.io/examples/nlp/lstm_seq2seq/

reference: https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
def load_data(path):
    with open(path) as fil:
        data = pd.read_csv(fil,sep='\t',header=None,names=["hi","en",""],skip_blank_lines=True,index_col=None)
    data = data[['hi','en']]
    return data

In [ ]:
# Download the dataset
!curl https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar --output daksh.tar
# Extract the downloaded tar file
!tar -xvf  'daksh.tar'
# Set the file paths to train, validation and test dataset
train_file=os.path.join(os.getcwd(),"dakshina_dataset_v1.0","hi","lexicons","hi.translit.sampled.train.tsv")
vaildation_file_path = os.path.join(os.getcwd(),"dakshina_dataset_v1.0","hi","lexicons","hi.translit.sampled.dev.tsv")
test_file_path = os.path.join(os.getcwd(),"dakshina_dataset_v1.0","hi","lexicons","hi.translit.sampled.test.tsv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915M  100 1915M    0     0   219M      0  0:00:08  0:00:08 --:--:--  203M
dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script

In [ ]:
train_1 = load_data("/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv")
dev_1 = load_data("/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv")
test_1 = load_data("/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv")

In [ ]:
train_1 = train_1[train_1['hi'].notna()]
train_1= train_1[train_1['en'].notna()]
dev_1 = dev_1[dev_1['hi'].notna()]
dev_1= dev_1[dev_1['en'].notna()]
test_1 = test_1[test_1['hi'].notna()]
test_1= test_1[test_1['en'].notna()]

In [ ]:
train_1.head()

,hi,en
0,अं,an
1,अंकगणित,ankganit
2,अंकल,uncle
3,अंकुर,ankur
4,अंकुरण,ankuran


In [ ]:
y = train_1['hi'].values
x = train_1['en'].values
y = '\t'+y+'\n'

In [ ]:
x

array(['an', 'ankganit', 'uncle', ..., 'hyensang', 'xuanzang', 'om'],
      dtype=object)

In [ ]:
y

array(['\tअं\n', '\tअंकगणित\n', '\tअंकल\n', ..., '\tह्वेनसांग\n',
       '\tह्वेनसांग\n', '\tॐ\n'], dtype=object)

In [ ]:
xxl=[xx for xx,yy in zip(x,y)]
english_char = set()
hindi_char = set()
for i in range(0,len(xxl)):
  w1=xxl[i]
  #print(w1)
  for l in str(w1):
    english_char.add(l)
print(english_char)             #the set of all the letters

yyl=[yy for xx,yy in zip(x,y)]

for i in range(0,len(yyl)):
  w2=yyl[i]
  #print(w1)
  for l in str(w2):
    hindi_char.add(l)
print(hindi_char)    

{'t', 'h', 'f', 'n', 'x', 'u', 's', 'd', 'z', 'o', 'v', 'c', 'y', 'b', 'l', 'a', 'i', 'm', 'q', 'e', 'k', 'w', 'g', 'p', 'r', 'j'}
{'द', 'घ', 'प', 'ओ', 'ख', 'क', 'ग', 'ो', 'ॉ', 'ऊ', 'न', 'भ', '\t', 'ँ', 'ल', 'ण', 'ह', 'आ', 'ा', 'ङ', 'ज', 'स', 'ष', 'त', 'झ', 'ऑ', 'फ', 'ं', 'च', '़', 'ठ', 'श', '्', 'ड', 'ौ', 'इ', 'ॅ', 'अ', 'ट', 'ए', '\n', 'छ', 'म', 'उ', 'ध', 'ई', 'व', 'ी', 'ब', 'ृ', 'र', 'ढ', 'ः', 'ऋ', 'थ', 'े', 'ऐ', 'ि', 'य', 'औ', 'ू', 'ॐ', 'ञ', 'ु', 'ै'}


In [ ]:
english_char

{'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
hindi_char

{'\t',
 '\n',
 'ँ',
 'ं',
 'ः',
 'अ',
 'आ',
 'इ',
 'ई',
 'उ',
 'ऊ',
 'ऋ',
 'ए',
 'ऐ',
 'ऑ',
 'ओ',
 'औ',
 'क',
 'ख',
 'ग',
 'घ',
 'ङ',
 'च',
 'छ',
 'ज',
 'झ',
 'ञ',
 'ट',
 'ठ',
 'ड',
 'ढ',
 'ण',
 'त',
 'थ',
 'द',
 'ध',
 'न',
 'प',
 'फ',
 'ब',
 'भ',
 'म',
 'य',
 'र',
 'ल',
 'व',
 'श',
 'ष',
 'स',
 'ह',
 '़',
 'ा',
 'ि',
 'ी',
 'ु',
 'ू',
 'ृ',
 'ॅ',
 'े',
 'ै',
 'ॉ',
 'ो',
 'ौ',
 '्',
 'ॐ'}

In [ ]:
dct_eng= [(ch,i+1) for i,ch in enumerate(english_char)]
english_char_map = dict(dct_eng)

dct_hin= [(ch,i+1) for i,ch in enumerate(hindi_char)]
hindi_char_map = dict(dct_hin)

In [ ]:
english_char_map

{'a': 16,
 'b': 14,
 'c': 12,
 'd': 8,
 'e': 20,
 'f': 3,
 'g': 23,
 'h': 2,
 'i': 17,
 'j': 26,
 'k': 21,
 'l': 15,
 'm': 18,
 'n': 4,
 'o': 10,
 'p': 24,
 'q': 19,
 'r': 25,
 's': 7,
 't': 1,
 'u': 6,
 'v': 11,
 'w': 22,
 'x': 5,
 'y': 13,
 'z': 9}

In [ ]:
num=0
hindi_char_map[" "] = num
english_char_map[" "] = num

In [ ]:
english_char_map

{' ': 0,
 'a': 16,
 'b': 14,
 'c': 12,
 'd': 8,
 'e': 20,
 'f': 3,
 'g': 23,
 'h': 2,
 'i': 17,
 'j': 26,
 'k': 21,
 'l': 15,
 'm': 18,
 'n': 4,
 'o': 10,
 'p': 24,
 'q': 19,
 'r': 25,
 's': 7,
 't': 1,
 'u': 6,
 'v': 11,
 'w': 22,
 'x': 5,
 'y': 13,
 'z': 9}

In [ ]:
len_m_eng= [len(i) for i in x]
max_eng_len = max(len_m_eng)
len_m_hin= [len(i) for i in y]
max_hin_len = max(len_m_hin)

In [ ]:
max_eng_len

20

In [ ]:
def process(data):
    x,y = data['en'].values, data['hi'].values
    y = "\t" + y + "\n"
    
    a = np.zeros((len(x),max_eng_len),dtype="float32")
    b = np.zeros((len(y),max_hin_len),dtype="float32")
    c = np.zeros((len(y),max_hin_len,len(hindi_char)+1),dtype="int")
    
    
    for i,(xx,yy) in enumerate(zip(x,y)):
        for j,ch in enumerate(xx):
            a[i,j] = english_char_map[ch]

        a[i,j+1:] = english_char_map[" "]
        for j,ch in enumerate(yy):
            b[i,j] = hindi_char_map[ch]

            if j>0:
                c[i,j-1,hindi_char_map[ch]] = 1

        b[i,j+1:] = hindi_char_map[" "]
        c[i,j:,hindi_char_map[" "]] = 1
        
    return a,b,c

In [ ]:
trainx, trainxx, trainy = process(train_1)
valx, valxx, valy = process(dev_1)
testx,testxx,testy = process(test_1)

In [ ]:
testx

array([[16.,  4., 21., ...,  0.,  0.,  0.],
       [16.,  4., 21., ...,  0.,  0.,  0.],
       [16.,  4., 21., ...,  0.,  0.,  0.],
       ...,
       [ 2., 10.,  7., ...,  0.,  0.,  0.],
       [ 2., 10.,  7., ...,  0.,  0.,  0.],
       [ 2., 10.,  7., ...,  0.,  0.,  0.]], dtype=float32)

In [ ]:
np.random.seed(25)

In [ ]:
rev_dst_eng= [(i,char) for char,i in english_char_map.items()]
reverse_eng_map = dict(rev_dst_eng)
rev_dst_hin= [(i,char) for char,i in hindi_char_map.items()]
reverse_hin_map = dict(rev_dst_hin)

In [ ]:
reverse_eng_map

{0: ' ',
 1: 't',
 2: 'h',
 3: 'f',
 4: 'n',
 5: 'x',
 6: 'u',
 7: 's',
 8: 'd',
 9: 'z',
 10: 'o',
 11: 'v',
 12: 'c',
 13: 'y',
 14: 'b',
 15: 'l',
 16: 'a',
 17: 'i',
 18: 'm',
 19: 'q',
 20: 'e',
 21: 'k',
 22: 'w',
 23: 'g',
 24: 'p',
 25: 'r',
 26: 'j'}

In [ ]:
import keras
import tensorflow as tf

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
from keras.layers import SimpleRNN,LSTM,GRU,Embedding,Dense,Dropout,Input
from tensorflow.keras.optimizers import Adam,Nadam
from keras import Model

In [ ]:
def GRU_model(cell = "GRU",nunits = 32, enc_layers = 1, dec_layers = 1,embed_dim = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_dim=len(english_char)+1,output_dim = embed_dim,mask_zero=True,name="enc_embed")
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(input_dim = len(hindi_char)+1,output_dim = embed_dim,mask_zero=True,name="dec_embed")
    decoder_context = decoder_embedding(decoder_inputs)

    if cell == "GRU":
        encoder_prev = [GRU(nunits,return_sequences=True,name=f"enc_{i}") for i in range(enc_layers-1)]
        encoder_fin = GRU(nunits,return_state=True,name=f"enc_{enc_layers-1}")
        temp = encoder_context
        for i,lay in enumerate(encoder_prev):
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout,name=f"do_{i}")(temp)
            
        _, state = encoder_fin(temp)
        encoder_states = state
        
        decoder = [GRU(nunits,return_sequences=True,return_state=True,name=f"dec_{i}") for i in range(dec_layers)]
        
        temp,s = decoder[0](decoder_context,initial_state=state)
        for i in range(1,dec_layers):
            temp,s = decoder[i](temp,initial_state=state)
            
        
    dense_lay1 = Dense(dense_size,activation='relu',name='dense1')
    pre_out = dense_lay1(temp)
    dense_lay2 = Dense(len(hindi_char)+1,activation = 'softmax',name='dense2')
    final_output = dense_lay2(pre_out)
    
    train = Model([encoder_inputs,decoder_inputs],final_output)
    
    return train


In [ ]:
def SimpleRNN(cell = "Simple",nunits = 32, enc_layers = 1, dec_layers = 1,embed_dim = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_dim=len(english_char)+1,output_dim = embed_dim,mask_zero=True,name="enc_embed")
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(input_dim = len(hindi_char)+1,output_dim = embed_dim,mask_zero=True,name="dec_embed")
    decoder_context = decoder_embedding(decoder_inputs)

    if cell == "Simple":
        encoder_prev = [SimpleRNN(nunits,return_sequences=True,name=f"enc_{i}") for i in range(enc_layers-1)]
        encoder_fin = SimpleRNN(nunits,return_state=True,name=f"enc_{enc_layers-1}")
        temp = encoder_context
        for i,lay in enumerate(encoder_prev):
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout,name=f"do_{i}")(temp)
            
        _, state = encoder_fin(temp)
        encoder_states = state
        
        decoder = [SimpleRNN(nunits,return_sequences=True,return_state=True,name=f"dec_{i}") for i in range(dec_layers)]
        
        temp,s = decoder[0](decoder_context,initial_state=state)
        for i in range(1,dec_layers):
            temp,s = decoder[i](temp,initial_state=state)

    dense_lay1 = Dense(dense_size,activation='relu',name='dense1')
    pre_out = dense_lay1(temp)
    dense_lay2 = Dense(len(hindi_char)+1,activation = 'softmax',name='dense2')
    final_output = dense_lay2(pre_out)
    
    train = Model([encoder_inputs,decoder_inputs],final_output)
    
    return train

In [ ]:
def lstm_model(cell = "LSTM",nunits = 32, enc_layers = 1, dec_layers = 1,embed_dim = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_dim=len(english_char)+1,output_dim = embed_dim,mask_zero=True,name="enc_embed")
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(input_dim = len(hindi_char)+1,output_dim = embed_dim,mask_zero=True,name="dec_embed")
    decoder_context = decoder_embedding(decoder_inputs)

    if cell == "LSTM":
        encoder_prev = [LSTM(nunits,return_sequences=True,name=f"enc_{i}") for i in range(enc_layers-1)]
        encoder_fin = LSTM(nunits,return_state=True,name=f"enc_{enc_layers-1}")
        temp = encoder_context
        for i,lay in enumerate(encoder_prev):
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout,name=f"do_{i}")(temp)
            
        _, state_h,state_c = encoder_fin(temp)
        encoder_states = [state_h,state_c]
        
        decoder = [LSTM(nunits,return_sequences=True,return_state=True,name=f"dec_{i}") for i in range(dec_layers)]
        
        temp,sh,sc = decoder[0](decoder_context,initial_state=encoder_states)
        for i in range(1,dec_layers):
            temp,sh,sc = decoder[i](temp,initial_state=encoder_states)
            
    
    dense_lay1 = Dense(dense_size,activation='relu',name='dense1')
    pre_out = dense_lay1(temp)
    dense_lay2 = Dense(len(hindi_char)+1,activation = 'softmax',name='dense2')
    final_output = dense_lay2(pre_out)
    
    train = Model([encoder_inputs,decoder_inputs],final_output)
    
    return train


In [ ]:
#for wandb
sweep_config = {
    'method': 'bayes', #grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'cell': {
            'values': ["LSTM","GRU","Simple"]
        },
        'embed_size': {
            'values': [32,128,256]
        },
        'lr': {
            'values': [1e-2,1e-3]
        },
        'dense_size': {
            'values': [64,128,512]
        },
        'dropout': {
            'values': [0.0,0.2,0.5]
        },
        'nunits': {
            'values': [128,256]
        },
        'enc_layers': {
            'values': [1,3,5]
        },
        'dec_layers': {
            'values': [1,3,5]
        },
        
    }
}


In [ ]:
#sweep_id = wandb.sweep(sweep_config,project="a=3", entity="dl22")

In [ ]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'lr': 1e-2,
        'dense_size': 64,
        'nunits': 128,
        'cell': 'LSTM',
        'embed_size': 64,
        'enc_layers': 1,
        'dec_layers': 1,
        'dropout': 0.
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults,name="cs6910")
    
    cfg = wandb.config
    
    name = f'nunits_{cfg.nunits}_cell_{cfg.cell}_encl_{cfg.enc_layers}_decl_{cfg.dec_layers}_emb_{cfg.embed_size}_ds_{cfg.dense_size}_do_{cfg.dropout}_lr_{cfg.lr}'
    wandb.run.name = name
    wandb.run.save()
    
    # Config is a variable that holds and saves hyperparameters and inputs
    
    train,enc,dec = lstm_model(nunits=cfg.nunits,
                                dense_size=cfg.dense_size,
                                enc_layers=cfg.enc_layers,
                                dec_layers=cfg.dec_layers,
                                cell = cfg.cell,
                                dropout = cfg.dropout,
                                embed_dim = cfg.embed_size)
    train.compile(optimizer = Adam(lr=cfg.lr),loss='categorical_crossentropy',metrics=['accuracy'])
    print("model building done")
    train.fit([trainx,trainxx],trainy,
             batch_size=64,
             validation_data = ([valx,valxx],valy),
             epochs=10,
             callbacks = [WandbCallback(monitor='val_accuracy',mode='min')])
    print("model training done")


In [ ]:
#wandb.agent(sweep_id, train,count=25)

In [ ]:
def accuracy1(real,pred):
    real = tf.math.argmax(real,axis=2)
    pred = tf.math.argmax(pred,axis=2)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    acc = tf.math.equal(real,pred)
    mask = tf.cast(mask, dtype='int32')
    acc = tf.cast(acc, dtype='int32')
    acc = tf.math.multiply(acc,mask)
    mask = tf.reduce_sum(mask,axis=1)
    acc = tf.reduce_sum(acc,axis=1)
    acc = tf.math.equal(acc,mask)
    acc = tf.cast(acc, dtype='float32')
    return tf.reduce_mean(acc)

In [ ]:
train = lstm_model(nunits=256,
                                dense_size=512,
                                enc_layers=3,
                                dec_layers=1,
                                cell = "LSTM",
                                dropout = 0.2,
                                embed_dim = 256)
train.compile(optimizer = Adam(lr=1e-3),loss='categorical_crossentropy',metrics=[accuracy1])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model_cb = tf.keras.callbacks.ModelCheckpoint('best_model.h5',monitor='val_accuracy1',mode='max',save_best_only=True,verbose=1)

In [ ]:
train.fit([trainx,trainxx],trainy,
             batch_size=64,
             validation_data = ([valx,valxx],valy),
             epochs=1,
             callbacks = [model_cb])

691/691 [==============================] - ETA: 0s - loss: 0.7499 - accuracy1: 0.0122
Epoch 1: val_accuracy1 improved from -inf to 0.07150, saving model to best_model.h5
691/691 [==============================] - 49s 32ms/step - loss: 0.7499 - accuracy1: 0.0122 - val_loss: 0.3929 - val_accuracy1: 0.0715


In [ ]:
model = keras.models.load_model('best_model.h5',custom_objects={'accuracy1':accuracy1})

In [ ]:
def GRU_inference(model,nunits=32,enc_layers=1,dec_layers=1,cell='LSTM',dropout=None):
    encoder_inputs = model.input[0]
    encoder_embedding = model.get_layer('enc_embed')
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = model.input[1]
    decoder_embedding = model.get_layer('dec_embed')
    decoder_context = decoder_embedding(decoder_inputs)
    
    
    encoder_prev = [model.get_layer(f'enc_{i}') for i in range(enc_layers-1)]
    encoder_fin = model.get_layer(f'enc_{enc_layers-1}')
    temp = encoder_context
    for i,lay in enumerate(encoder_prev):
        temp = lay(temp)
        if dropout is not None:
            temp = model.get_layer(f'do_{i}')(temp)
    decoder = [model.get_layer(f'dec_{i}') for i in range(dec_layers)]
    if cell == "GRU":
        state_inputs = []
        state_outputs = []

        state_input = Input(shape=(nunits,),name='inputs0')
        temp,s = decoder[0](decoder_context,initial_state = state_input)
        state_inputs.append(state_input)
        state_outputs.append(s)

        for i in range(1,dec_layers):
            state_input = Input(shape=(nunits,),name=f'inputs{i}')
            temp,s = decoder[i](temp,initial_state = state_input)
            state_inputs.append(state_input)
            state_outputs.append(s)

        decoder_input_pass = [decoder_inputs] + state_inputs

    pre_out = model.get_layer('dense1')(temp)
    final_output = model.get_layer('dense2')(pre_out)

    decoder_model = keras.models.Model(decoder_input_pass, [final_output]+state_outputs)
    
    return encoder_model,decoder_model

In [ ]:
def lstm_inference(model,nunits=32,enc_layers=1,dec_layers=1,cell='LSTM',dropout=None):
    encoder_inputs = model.input[0]
    encoder_embedding = model.get_layer('enc_embed')
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = model.input[1]
    decoder_embedding = model.get_layer('dec_embed')
    decoder_context = decoder_embedding(decoder_inputs)
    encoder_prev = [model.get_layer(f'enc_{i}') for i in range(enc_layers-1)]
    encoder_fin = model.get_layer(f'enc_{enc_layers-1}')
    temp = encoder_context
    for i,lay in enumerate(encoder_prev):
        temp = lay(temp)
        if dropout is not None:
            temp = model.get_layer(f'do_{i}')(temp)
    if cell == "LSTM":
        _, state_h,state_c = encoder_fin(temp)
        encoder_states = [state_h,state_c]
    encoder_model = keras.models.Model(encoder_inputs,encoder_states)
    
    
    
    decoder = [model.get_layer(f'dec_{i}') for i in range(dec_layers)]
    if cell == "LSTM":
        state_inputs = []
        state_outputs = []

        decoder_input_h = Input(shape=(nunits,),name='inputh0')
        decoder_input_c = Input(shape=(nunits,),name='inputc0')
        temp,sh,sc = decoder[0](decoder_context,initial_state = [decoder_input_h,decoder_input_c])
        state_inputs += [decoder_input_h,decoder_input_c]
        state_outputs += [sh,sc]

        for i in range(1,dec_layers):
            decoder_input_h = Input(shape=(nunits,),name=f'inputh{i}')
            decoder_input_c = Input(shape=(nunits,),name=f'inputc{i}')
            temp,sh,sc = decoder[i](temp,initial_state = [decoder_input_h,decoder_input_c])
            state_inputs += [decoder_input_h,decoder_input_c]
            state_outputs += [sh,sc]

        decoder_input_pass = [decoder_inputs] + state_inputs

    pre_out = model.get_layer('dense1')(temp)
    final_output = model.get_layer('dense2')(pre_out)

    decoder_model = keras.models.Model(decoder_input_pass, [final_output]+state_outputs)
    
    return encoder_model,decoder_model


In [ ]:
enc,dec = lstm_inference(model,nunits=256,enc_layers=3,dec_layers=1,cell="LSTM",dropout='yes')

In [ ]:
enc.save('best_enc.h5')
dec.save('best_dec.h5')